In [5]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

# Load dataset & Define image directory
train_df = pd.read_csv('/Users/annahauk/Desktop/bttai-nybg-2024/BTTAIxNYBG-test.csv')  
validate_df = pd.read_csv('/Users/annahauk/Desktop/bttai-nybg-2024/BTTAIxNYBG-validation.csv')

image_directory = '/Users/annahauk/Desktop/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'

# Preprocessing function to load and process images
def load_and_preprocess_image(filename, target_size=(224, 224)):
    img_path = os.path.join(image_directory, filename)
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Model expects a batch of images
    return img_array / 255.0  # Normalize to [0, 1]


In [6]:
# Apply preprocessing to all images
train_df['imageData'] = train_df['imageFile'].apply(load_and_preprocess_image)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/annahauk/Desktop/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train/facd4dcd8e869617.jpg'

In [ ]:
# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note: No augmentation for validation data, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# Convert dataframe to a format suitable for the model training
def df_to_dataset(dataframe, datagen, batch_size=32):
    datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=image_directory,
        x_col='imageFile',
        y_col='classLabel',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'  # Change this if not a multiclass classification
    )

# Create datasets for training and validation
train_dataset = df_to_dataset(train_df, train_datagen)
validation_dataset = df_to_dataset(validate_df, validation_datagen)

# This setup is now ready for training with model.fit using the train_dataset and validation_dataset